In [1]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

import csv, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

pd.set_option('display.max_columns', 500)
curr_dir = '/home/souche/qiongjiu/hgc/'

In [2]:
data = pd.read_csv(curr_dir+"2020-11-02版带count处理完成的采购数据m.csv", header = 0, low_memory=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399647 entries, 0 to 1399646
Data columns (total 61 columns):
car_id                      1399647 non-null int64
brand_code                  1399543 non-null object
brand_name                  1399543 non-null object
series_code                 1399543 non-null object
series_name                 1399543 non-null object
model_code                  1399647 non-null object
model_name                  1399543 non-null object
mileage_std                 1399647 non-null float64
color_name                  1399405 non-null object
first_license_plate_date    1399647 non-null object
province_name               1399647 non-null object
city_name                   1399647 non-null object
purchase_price              1399647 non-null float64
order_create_time           1399647 non-null object
license_month               1399647 non-null int64
residual                    1399647 non-null float64
days                        1399647 non-null int64
lic

In [3]:
car = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header = 0, low_memory=False)
car=car.drop(['model_year','new_car_price', 'driving_mode','gear_box_type','country_id',
 'import_type','intake_type', 'fuel_form','quality_mile', 'quality_year',
 'year_1','year_2','year_3','year_4','year_5','year_6','year_7','year_8',
 'year_9', 'year_10','year_11','year_12','year_13','year_14','year_15',
 'year_16', 'car_body','rate','rate_count'],axis=1)

print(car.shape)


(65250, 151)


In [4]:
data = pd.merge(data, car, how="inner", on="model_code")

vec = pd.read_csv(curr_dir+"2020-11-02版-TF-IDF权重的词向量.csv", dtype={'model_code': str}, header = 0)
vec = vec.drop(['model_name'], axis=1)
print(vec.shape)

data = pd.merge(data, vec, how="inner", on="model_code")
print(data.shape)

(65250, 101)
(1399647, 311)


In [6]:
data_B = data[(data['level']=="B") | 
              (data['level']=="C") | 
              (data['level']=="D") | 
              (data['level']=="E")]
print("筛选出B级以上的车数据量为:%d" % data_B.shape[0])
print("筛选出B级以上的车的model_code为:%d" % len(data_B['model_code'].unique()))
print("包含的车型级别:", data_B['level'].unique())

筛选出B级以上的车数据量为:668434
筛选出B级以上的车的model_code为:9851
包含的车型级别: ['B' 'C' 'D' 'E']


In [7]:
data_X = data_B.drop(['car_id', 'brand_code', 'brand_name', 'series_code',
                      'series_name', 'model_code', 'model_name',  'color_name',
                      'first_license_plate_date', 'province_name', 'city_name',
                      'purchase_price', 'order_create_time', 'residual', 'mileage_std', 'license_month',
                      'new_car_price', 'count', 'series_level', 'level', 'level0', 'level1',
                      'level2', 'level3', 'level4', 'level5', 'level6', 'level7', 'level8'], axis=1)
data_y = data_B['residual']

print(data_X.shape, len(data_y))

(668434, 282) 668434


In [8]:
import lightgbm as lgb

gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression', 
                        learning_rate = 0.04, 
                        n_estimators = 2800, 
                        max_depth = 8, 
                        num_leaves = 150, 
                        subsample = 0.95, 
                        colsample_bytree = 0.3, 
                        min_child_samples = 100, 
                        n_jobs = 8, 
                        verbose = 1)
gbm.fit(data_X, 
        data_y,
        verbose = 2)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3,
       learning_rate=0.04, max_depth=8, min_child_samples=100,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2800,
       n_jobs=8, num_leaves=150, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.95,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [13]:
data_B['ypred'] = gbm.predict(data_X)
data_B['err'] = abs(data_B['ypred'] - data_B['residual'])/data_B['residual']
data_B[data_B['err']>=0.1].shape

(133682, 314)

In [14]:
def data3m_pinggu(data_df,col):
    total = data_df.shape[0]
    num_3 = data_df.loc[(data_df[col]>=-0.03 ) & (data_df[col]<=0.03 )].shape[0]
    print("P<3%: ",round(num_3/total,4))
    num_5 = data_df.loc[(data_df[col]>=-0.05 ) & (data_df[col]<=0.05 )].shape[0]
    print("P<5%: ",round(num_5/total,4))
    num_8 = data_df.loc[(data_df[col]>=-0.08 ) & (data_df[col]<=0.08 )].shape[0]
    print("P<8%: ",round(num_8/total,4))
    num_10 = data_df.loc[(data_df[col]>=-0.1 ) & (data_df[col]<=0.1 )].shape[0]
    print("P<10%: ",round(num_10/total,4))
    num_20 = data_df.loc[(data_df[col]>=-0.2 ) & (data_df[col]<=0.2 )].shape[0]
    print("P<20%: ",round(num_20/total,4))

data_B['差异'] = (data_B['ypred']-data_B['residual'])/data_B['residual']
data3m_pinggu(data_B,'差异')

P<3%:  0.3621
P<5%:  0.5428
P<8%:  0.723
P<10%:  0.8
P<20%:  0.9462


In [12]:
data_B = data_B[data_B['err']<0.1]
#打乱数据集
data_B = shuffle(data_B)

data_X = data_B.drop(['car_id', 'brand_code', 'brand_name', 'series_code',
                      'series_name', 'model_code', 'model_name',  'color_name',
                      'first_license_plate_date', 'province_name', 'city_name',
                      'purchase_price', 'order_create_time', 'residual', 'mileage_std', 'license_month',
                      'new_car_price', 'count', 'series_level', 'level', 'level0', 'level1',
                      'level2', 'level3', 'level4', 'level5', 'level6', 'level7', 'level8', 'ypred', 'err'], axis=1)
data_y = data_B['residual']
print(data_X.shape, len(data_y))

(653554, 282) 653554


In [13]:
for i in range(15):
    data_X = data_X.rename(columns={"country_id"+str(i):"Country_Id"+str(i)})


In [15]:
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
import lightgbm as lgb

pipeline = PMMLPipeline([("regressor", lgb.LGBMRegressor(boosting_type='gbdt',
                                                         objective='regression',
                                                         learning_rate=0.04,
                                                         n_estimators=2800,
                                                         max_depth=8,
                                                         num_leaves=150,
                                                         subsample=0.95,
                                                         colsample_bytree=0.3,
                                                         min_child_samples=100,
                                                         n_jobs=8,
                                                         verbose=1))])
pipeline.fit(data_X, data_y)
ph = curr_dir+"enterprise2_BBB_level_dfc_purchase.pmml"
sklearn2pmml(pipeline, ph, with_repr=True)

In [16]:
joblib.dump(pipeline,curr_dir+'B_level_dfc_purchase_model_2020-11-02')


['/home/souche/qiongjiu/hgc/B_level_dfc_purchase_model_2020-11-02']

In [17]:
# 上传到 oss上 
ph = curr_dir+"enterprise2_BBB_level_dfc_purchase.pmml"
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/2020-11-02/enterprise2_BBB_level_dfc_purchase.pmml", ph)

DAYU_HOME : /home/souche/projects/datacenter-etl-v2
[2020-11-05 12:30:14,296] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
[2020-11-05 12:30:14,325] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
[2020-11-05 12:30:15,540] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation/2020-11-02/enterprise2_BBB_level_dfc_purchase.pmml .
